In [1]:
import os; os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import h5py as h5
import numpy as np
import pycuda.driver as cuda
import pycuda.gpuarray as ga

from chroma.event import Photons
import h5py as h5

f = h5.File("/sdf/data/neutrino/public_html/dc/mpvmpr_20.h5", "r", libver="latest", swmr=True)

INVALID_TRACK_ID = pow(2, 31) - 1
light_yield = 20000 # photons/MeV @ 500V/cm

def photons(f, wavelength=450):
    """
    note: wavelength doesn't matter as there are no wavelength-dependent effects
    during propagation in the liquid argon. however, there *are* in the glass of the PMTs,
    so we set the wavelength to a wavelength similar to that after being reemitted by TPB.
    """
    events_s_v = f['pstep/lar_vol']
    events_p_v = f['particle/geant4']
    for i in range(len(events_s_v)):
        event_s = events_s_v[i]
        event_p = events_p_v[i]
        mask = ~(event_s["track_id"] == INVALID_TRACK_ID)
        event_s = event_s[mask]

        # for computing beta*gamma
        # mass = event_p['mass'][event_s['track_id']]
        # mask = mass > 0
        # event_s = event_s[mask]
        # mass = mass[mask]

        de = event_s['de']
        t = event_s['t']
        x, y, z = event_s['x'], event_s['y'], event_s['z']
        pos = np.column_stack([x, y, z])

        num_photons = np.ceil(de * light_yield).astype(np.int32)
        total_photons = np.sum(num_photons)

        # random direction
        costheta = np.random.random(total_photons) * 2 - 1
        sintheta = np.sqrt(1 - np.square(costheta))
        phi = np.random.random(total_photons) * 2 * np.pi
        cosphi = np.cos(phi)
        sinphi = np.sin(phi)
        pdir = np.transpose([sintheta * cosphi, sintheta * sinphi, costheta])

        # random polarization
        costheta = np.random.random(total_photons) * 2 - 1
        sintheta = np.sqrt(1 - np.square(costheta))
        phi = np.random.random(total_photons) * 2 * np.pi
        cosphi = np.cos(phi)
        sinphi = np.sin(phi)
        rand_unit = np.transpose([sintheta * cosphi, sintheta * sinphi, costheta])
        ppol = np.cross(pdir, rand_unit)
        ppol = ppol / np.linalg.norm(ppol, ord=2, axis=1, keepdims=True)

        if type(wavelength) is tuple:
            pwavelength = (
                np.random.random(total_photons) * (wavelength[1] - wavelength[0]) + wavelength[0]
            )
        else:
            pwavelength = np.tile(wavelength, total_photons)

        photons = Photons(
            pos=np.repeat(pos, num_photons, axis=0),
            dir=pdir,
            pol=ppol,
            t=np.repeat(t, num_photons),
            wavelengths=pwavelength,
            evidx=np.repeat(i, total_photons),
        )
        yield photons


num_photons = [(f['pstep/lar_vol'][i]['de']*light_yield).sum() for i in range(len(f['pstep/lar_vol']))]
num_photons

[18528768.0,
 156654200.0,
 154819740.0,
 55836772.0,
 185839600.0,
 154153660.0,
 135658200.0,
 97843944.0,
 53128124.0,
 132014480.0,
 178409970.0,
 234665250.0,
 196912220.0,
 165495200.0,
 154901840.0,
 201038110.0,
 216578430.0,
 124699110.0,
 180451780.0,
 60925984.0]

In [3]:
from chroma_lar.geometry import build_detector_from_config
from chroma.sim import Simulation
import time

cfg = dict(
    include_active=True,
    include_cavity=False,
    include_pmts=True,
    include_cathode=True,
    include_wires=True,
    analytic_wires=True,
)

g = build_detector_from_config("detector_config_reflect_reflect3wires", **cfg)

sim = Simulation(g, photon_tracking=0)

# generate a small test of photons near a wireplane

# run a short propagation to get rid of overhead
from chroma_lar.generator import photon_gen
phot = photon_gen(10)
_ = list(
    sim.simulate(
        phot,
        keep_photons_end=False,
        keep_hits=False,
        keep_flat_hits=False,
        photons_per_batch=10,
        max_steps=1000,
    )
)

t_total = 0
t_start = time.time()
for i, ev in enumerate(
    sim.simulate(
            photons(f),
            keep_photons_end=False,
            keep_hits=False,
            keep_flat_hits=True,
            photons_per_batch=5_000_000,
            max_steps=1000,
        )
    ):
    print(f"event {i}/{len(f['pstep/lar_vol'])}")

    t = ev.flat_hits.t # ...

    t_end = time.time()
    t_total += t_end - t_start
    t_diff = t_end - t_start
    t_start = t_end
    phot_sec = num_photons[i] / t_diff
    print(f"time: {t_total:.2f} s, {phot_sec/1_000:,.0f}k ph/s, {num_photons[i]:,.0f} ph")
print(f"total time: {t_total} seconds")


Total PMT number is 162
Spacing buffer in y: 320.0, z: 320.0
analytic wireplanes attached: 6 (total=6)
analytic wireplanes attached: 6
event 0/20
time: 19.76 s, 938k ph/s, 18,528,768 ph


event 1/20
time: 151.71 s, 1,187k ph/s, 156,654,208 ph


event 2/20
time: 287.45 s, 1,141k ph/s, 154,819,744 ph


KeyboardInterrupt: 